<a href="https://colab.research.google.com/github/fabiorfc/Epidemiologia-Matematica/blob/master/COVID_19_Tratamento_da_base_do_SUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tratamento da base de dados

___
## 1) Escopo do notebook

* Preparação do ambiente
* Visualização prévia dos dados
* Filtragem dos dados
* Estimativas a partir do número de óbitos
* Curvas de demandas por UTI
* Gravação da base de dados

___
## 2) Preparação do ambiente


**Importação das libraries**

In [1]:
import pandas as pd
from math import log,isnan
import matplotlib.pyplot as plt
from datetime import datetime,timedelta

**Funções**


In [2]:
def calculador_de_estimativas(cidades):
  """Funcao para calcular as estimativas necessarias para o trabalho"""
  tabela = cidades.copy()
  # Estimativas de infectados
  tabela.insert(loc = tabela.shape[1], column = 'infectados', value = [i / 0.00504 for i in tabela["obitosAcumulado"]])
  tabela.insert(loc = tabela.shape[1], column = 'infectados_log', value = [0 if i == 0 else log(i) for i in tabela["infectados"]])
  tabela.insert(loc = tabela.shape[1], column = 'infectados_uti', value = [i * 0.01960 for i in tabela["infectados"]])
  # Data estimada de adoecimento
  data = tabela['data']
  data_estimada = [i+timedelta(-7) for i in data]
  tabela.insert(loc = tabela.shape[1], column = 'data_estimada_acometimento', value = data_estimada)
  # Calculo das taxas de letalidade
  letalidade_01 = tabela['obitosAcumulado'] / tabela['casosAcumulado']
  tabela.insert(loc = tabela.shape[1], column = 'letalidade_01', value = letalidade_01)
  # Prevalencia estimada
  prevalencia = tabela['casosAcumulado'] / pd.to_numeric(tabela['populacaoTCU2019']) * 100000
  tabela.insert(loc = tabela.shape[1], column = 'prevalencia', value = prevalencia)
  # Incidência
  incidencia = tabela['casosNovos'] / pd.to_numeric(tabela['populacaoTCU2019']) * 100000
  tabela.insert(loc = tabela.shape[1], column = 'incidencia', value = incidencia)

  return tabela

___
## 3) Leitura e preparação dos dados

### 3.1) Base de dados do SUS

[Fonte de dados](https://covid.saude.gov.br/)

Base de dados que contém a incidência de casos e mortes em valores acumulados.

Colunas:
* Date: Data do registro
* Region: Região do registro
* State: Estado do registro
* Cases: Número acumulado de casos
* Deaths: Número acumulado de mortes

In [4]:
dados = pd.read_excel('HIST_PAINEL_COVIDBR_15jul2020.xlsx')
dados.tail(2)

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
611477,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2020-07-14,29,3015268,73654,1370,960,30,NaN,NaN,1.0
611478,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2020-07-15,29,3015268,75379,1725,1001,41,NaN,NaN,1.0


In [5]:
dados.drop(['coduf','codmun','codRegiaoSaude','emAcompanhamentoNovos','interior/metropolitana'], axis = 1, inplace = True)

### 3.2) Filtragem dos dados

In [13]:
estado_de_sao_paulo = dados.query("estado == 'SP' ")
cidade_de_sao_paulo = estado_de_sao_paulo.query("municipio == 'São Paulo'")
cidade_de_assis = estado_de_sao_paulo.query("municipio == 'Assis'")

print("Contabilização da base de dados:")
print("--- Tamanho da base geral: {}".format(len(dados)))
print("--- Tamanho da base do estado São Paulo: {}".format(len(estado_de_sao_paulo)))
print("--- Tamanho da base da cidade São Paulo: {}".format(len(cidade_de_sao_paulo)))
print("--- Tamanho da base da cidade Assis: {}".format(len(cidade_de_assis)))

Contabilização da base de dados:
--- Tamanho da base geral: 611479
--- Tamanho da base do estado São Paulo: 71071
--- Tamanho da base da cidade São Paulo: 111
--- Tamanho da base da cidade Assis: 111


In [14]:
cidade_de_assis.tail(2)

,regiao,estado,municipio,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos
370940,Sudeste,SP,Assis,ASSIS,2020-07-14,29,104386,348,42,14,0,NaN
370941,Sudeste,SP,Assis,ASSIS,2020-07-15,29,104386,353,5,14,0,NaN


In [15]:
cidade_de_sao_paulo.tail(2)

,regiao,estado,municipio,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos
427550,Sudeste,SP,São Paulo,SAO PAULO,2020-07-14,29,12252023,158802,4789,8336,105,NaN
427551,Sudeste,SP,São Paulo,SAO PAULO,2020-07-15,29,12252023,160326,1524,8451,115,NaN


___
## 5) Estimativas a partir do número de óbitos

Estima-se que 0,504% e 1,96% dos infectados morrem e precisam de uma unidade de internação, respectivamente

As taxas de incidência e mortalidade aqui apresentadas são baseadas nos cálculos apresentados em [Pan American Health Organization](https://www.paho.org/hq/index.php?option=com_content&view=article&id=14402:health-indicators-conceptual-and-operational-considerations-section-2&Itemid=0&limitstart=2&lang=pt)


1. **Estimativa do número de infectados**

\begin{equation}
infectados = \frac{mortes}{0.00504} \tag{1}
\end{equation}

\

2. **Estimativa do número de infectados com necessidade de UTI**

\

\begin{equation}
infectados-com-necessidade-de-uti = 0.0196 * infectados \tag{2}
\end{equation}

\

3. **Incidência**

Define-se como cidência pode ser interpretada como a probabilidade de um indivíduo pertencente à população de risco ser acometido pelo patógeno

\

\begin{equation}
incidencia = \frac{novos casos}{população De Risco} * 100.000
\end{equation}

\

4. **Prevalência**

A taxa de prevalência é definida como o número de indivíduos acometidos pelo patógeno, dividida pela população. Essa taxa pode ser interpretada como a probabilidade de um invíduo pertencente à população-base seja afetado pelo patógeno

\

\begin{equation}
prevalencia = \frac{total de casos}{populacao base} * 100.000
\end{equation}

\

4. **Cálculo da letalidade do vírus**

\begin{equation}
letalidade = \frac{mortes}{mortes + recuperados} \tag{3}
\end{equation}

* **Cálculo da letalidade estimada do vírus**

\begin{equation}
letalidade = \frac{mortes}{Infectados} \tag{3}
\end{equation}


### 5.1) Estimativas para a cidade de São Paulo

In [16]:
cidade_de_sao_paulo = calculador_de_estimativas(cidade_de_sao_paulo)
cidade_de_sao_paulo.tail(2)

,regiao,estado,municipio,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,infectados,infectados_log,infectados_uti,data_estimada_acometimento,letalidade_01,prevalencia,incidencia
427550,Sudeste,SP,São Paulo,SAO PAULO,2020-07-14,29,12252023,158802,4789,8336,105,NaN,1.653968e+06,14.318688,32417.777778,2020-07-07,0.052493,1296.128811,39.087423
427551,Sudeste,SP,São Paulo,SAO PAULO,2020-07-15,29,12252023,160326,1524,8451,115,NaN,1.676786e+06,14.332389,32865.000000,2020-07-08,0.052711,1308.567573,12.438762


### 5.2) Estimativas para a cidade de Assis

In [17]:
cidade_de_assis = calculador_de_estimativas(cidade_de_assis)
cidade_de_assis.tail(2)

,regiao,estado,municipio,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,infectados,infectados_log,infectados_uti,data_estimada_acometimento,letalidade_01,prevalencia,incidencia
370940,Sudeste,SP,Assis,ASSIS,2020-07-14,29,104386,348,42,14,0,NaN,2777.777778,7.929407,54.444444,2020-07-07,0.04023,333.378039,40.235281
370941,Sudeste,SP,Assis,ASSIS,2020-07-15,29,104386,353,5,14,0,NaN,2777.777778,7.929407,54.444444,2020-07-08,0.03966,338.167954,4.789914


## 6) Gravação da base de dados


In [18]:
cidade_de_sao_paulo.to_csv('sao_paulo.csv', index = False)
cidade_de_assis.to_csv('assis.csv', index = False)